# 01 Import Libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

# 02 Import Data

In [2]:
# Create path
path = r'/Users/dana/Documents/Instacart Basket Analysis/'

In [3]:
# import data
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))

# 03 Task

In [4]:
# find the aggregated mean of the “order_number” column grouped by “department_id” for the whole dataframe
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1.0,15.457838
2.0,17.277920
3.0,17.170395
4.0,17.811403
5.0,15.215751
6.0,16.439806
7.0,17.225802
8.0,15.340650


In [5]:
# the difference is that this includes all the department ids and the means are a little different

#### Create a loyalty flag

In [6]:
# make a max order column using transform 
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [7]:
# create a flag that assigns a loyalty label to each customer
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [9]:
# look up prices spent according to customer's loyalty
ords_prods_merge.groupby('loyalty_flag')['prices'].mean()

loyalty_flag
Loyal customer      10.386336
New customer        13.294670
Regular customer    12.495717
Name: prices, dtype: float64

#### Create a spender flag

In [11]:
# create an average spent by customer column
ords_prods_merge['avg_spent'] = ords_prods_merge.groupby(['user_id'])['prices'].mean()

In [12]:
# create a spender flag
ords_prods_merge.loc[ords_prods_merge['avg_spent'] < 10, 'spender_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['avg_spent'] >= 10, 'spender_flag'] = 'High spender'

In [16]:
# check results
ords_prods_merge[['user_id', 'avg_spent', 'spender_flag']].head(10)

,user_id,avg_spent,spender_flag
0,1.0,NaN,NaN
1,1.0,6.367797,Low spender
2,1.0,7.515897,Low spender
3,1.0,8.197727,Low spender
4,1.0,8.205556,Low spender
5,1.0,9.189189,Low spender
6,1.0,8.471429,Low spender
7,1.0,7.466990,Low spender
8,1.0,8.781633,Low spender
9,1.0,7.607895,Low spender


In [19]:
ords_prods_merge['spender_flag'].value_counts()

Low spender     200852
High spender      5357
Name: spender_flag, dtype: int64

#### Create an order frequency flag

In [14]:
# create an average spent by customer column
ords_prods_merge['order_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_last_order'].median()

In [22]:
# create a flag that assigns a loyalty label to each customer
ords_prods_merge.loc[ords_prods_merge['order_frequency'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['order_frequency'] <= 20) & (ords_prods_merge['order_frequency'] > 10), 'order_frequency_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['order_frequency'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [23]:
# check results
ords_prods_merge[['user_id', 'order_frequency', 'order_frequency_flag']].head(10)

,user_id,order_frequency,order_frequency_flag
0,1.0,NaN,NaN
1,1.0,20.5,Non-frequent customer
2,1.0,13.0,Regular customer
3,1.0,10.0,Frequent customer
4,1.0,20.0,Regular customer
5,1.0,11.0,Regular customer
6,1.0,6.0,Frequent customer
7,1.0,9.0,Frequent customer
8,1.0,30.0,Non-frequent customer
9,1.0,30.0,Non-frequent customer


In [24]:
ords_prods_merge['order_frequency_flag'].value_counts()

Frequent customer        86591
Regular customer         60002
Non-frequent customer    59616
Name: order_frequency_flag, dtype: int64

# 04 Export Data

In [25]:
# export dataframe
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merge.pkl'))